PCA plots - Ally - *PENDING*
    - already implemented in R, just need to plot it in Hail
    
----------------------------------------
Further edits needed in this nb: 
- Add the path to the PCA plotting Rmarkdown (in section 5) once available  
- Complete the table in section 5 - need Alicia's help with that 
- Add Ally's code for plots 

# PCA and Ancestry Analyses


Author: Mary T. Yohannes

# Index
1. [Setting Default Paths](#1.-Set-Default-Paths)
2. [Annotating Plotting Metadata onto Matrix Table](#-2.-Annotating-Plotting-Metadata)
3. [Variant Filtering and LD Pruning](#3.-Variant-Filtering-and-LD-Pruning)
    1. [Variant Filtering](#-3a.-Variant-Filtering)
    2. [LD Pruning](#-3b.-LD-Pruning)
    3. [Write out an intermediate file](#-3c.-Write-out-an-intermediate-file)
4. [Run PC Relate](#4.-Run-PC-Relate)
5. [PCA](#5.-PCA)
    1. [Function to Run PCA on Unrelated Individuals](#5a.-Function-to-Run-PCA-on-Unrelated-Individuals)
    2. [Function to Project Related Individuals](#5b.-Function-to-Project-Related-Individuals)
    3. [Global PCA](#5c.-Global-PCA)
    4. [Subcontinental PCA](#5d.-Subcontinental-PCA)
6. [Outlier Removal](#6.-Outlier-Removal)
7. [Rerun PCA](#7.-Rerun-PCA)
    1. [Global PCA](#7a.-Global-PCA)
    2. [Subcontinental PCA](#7b.-Subcontinental-PCA)
8. [Writing out Matrix Table](#8.-Write-Out-Matrix-Table)

# General Overview 

The purpose of this notebook is to further filter the postQC Matrix Table to prepare it for LD pruning, compute relatedness and run Principal Component Analysis (PCA).

**This script contains information on how to:**
- Read in the a Matrix Table and run Hail common variant statistics  
- Filter using allele frequency and call rate
- Run LD pruning 
- Run relatedness and separate related and unrelated individuals
- Calculate PC scores and project samples on to a PC space  
- Run global and Subcontinental PCA and plot them 
- Remove PCA outliers (filter using sample IDs)
- Rerun global and subcontinental PCA
- Write out a Matrix Table 


In [ ]:
import hail as hl

# Importing methods from gnomAD needed to project individuals
from gnomad.sample_qc.ancestry import *

# 1. Set Default Paths
These default paths can be edited by users as needed. It is recommended to run these tutorials without writing out datasets. 

Certain steps such as [LD Pruning](#2.-Variant-Filtering-and-LD-Pruning) and [`pc_relate`](#3.-Run-PC-Relate) can take around 2 hours to run. We have written out datasets at these steps so users can run the tutorials without waiting for these steps to complete.  

By default all of the write sections are shown as markdown cells. If you would like to write out your own datasets, you can copy the code and paste it into a new code cell. 

[Back to Index](#Index)

In [ ]:
# Path for dataset after applying gnomAD's sample and variant QC
post_qc_path = 'gs://hgdp-1kg/hgdp_tgp/intermediate_files/pre_running_varqc.mt'

# Path to save the filtered and LD pruned mt as an intermediate file since LD pruning takes a while to rerun
intermediate_file_path = 'gs://hgdp-1kg/hgdp_tgp/intermediate_files/filtered_n_pruned_output_updated.mt'

# Path for the metadata file which contains colors for PCA plotting
plotting_metadata_path = 'gs://hgdp-1kg/hgdp_tgp/qc_and_figure_generation/hgdp_1kgp_plotting_meta.tsv'

# Paths for unrelated and related samples (prior to outlier identification and removal) 
unrel_preoutlier_path = 'gs://hgdp-1kg/hgdp_tgp/unrel_updated.mt'
rel_preoutlier_path = 'gs://hgdp-1kg/hgdp_tgp/rel_updated.mt' 

# Path for pre-outlier PCA results - global & subcontinental PCA 
pca_preoutlier_path = 'gs://hgdp-1kg/hgdp_tgp/pca_preoutlier/'

# Outliers file 
outliers_path = 'gs://hgdp-1kg/hgdp_tgp/pca_outliers_v2.txt'

# Path for post-outlier PCA results - global & subcontinental PCA 
pca_postoutlier_path = 'gs://hgdp-1kg/hgdp_tgp/pca_postoutlier/'

# Final output paths for unrelated and related samples (post-outlier)
unrel_final_output = 'gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/unrelated.mt'
rel_final_output = 'gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/related.mt'

# 2. Annotating Plotting Metadata
Below we are adding information from a metadata file which contains information on which shapes and colors to use for each population when plotting PCA

In [ ]:
# Reading in the plotting metadata file
plotting_meta = hl.read_table(plotting_metadata_path)

# Reading in the dataset after applying gnomAD's sample and variant QC filters
post_qc = hl.read_matrix_table(post_qc_path)

# Using hl.annotate_cols() method to annotate the plotting metadata onto the Matrix Table
mt_filt = post_qc.annotate_cols(**plotting_meta[post_qc.s])

# 3. Variant Filtering and LD Pruning
<br>
<details><summary> Click <u><span style="color:blue">here</span></u> to learn why we are doing this. </summary>
    
At this point, we have 155,648,020 SNPs and since we need fewer number of variants (~100-300k) for PCA, we filter on:
- AF - allele frequency 
- call rate - fraction of calls neither missing nor filtered

and then run LD pruning.     
  
Linkage disequilibrium (LD) is the correlation between nearby variants such that the alleles at neighboring polymorphisms (observed on the same chromosome) are associated within a population more often than if they were unlinked.
<br>    
For more information on LD pruning click <a href=""> here </a>
</details>

<br>
<details><summary> For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
<ul>
<li><a href="https://hail.is/docs/0.2/methods/genetics.html#hail.methods.variant_qc"> More on  <i> variant_qc() </i></a></li>

<li><a href="https://hail.is/docs/0.2/methods/genetics.html#hail.methods.ld_prune"> More on  <i> ld_prune() </i></a></li>
    </ul>
</details>

[Back to Index](#Index)

#### 3a. Variant Filtering 

In [ ]:
# Run Hail's common variant statistics (QC metrics) 
mt_var = hl.variant_qc(mt_filt) 

# Filter to variants with AF between 0.05 & 0.95, and call rate greater than 0.999    
mt_var_filt = mt_var.filter_rows((mt_var.variant_qc.AF[0] > 0.05) & 
                                 (mt_var.variant_qc.AF[0] < 0.95) & (mt_var.variant_qc.call_rate > 0.999))

# Should print 6787034 snps; this line take ~20min to run 
print('Num of variants after filtering = ' + str(mt_var_filt.count()[0]))

#### 3b. LD Pruning
Since the number of variants after this step is now in the ~100-300k range, we proceed to the PCA analysis without any more adjustments.  

In [ ]:
# Remove correlated variants 
pruned = hl.ld_prune(mt_var_filt.GT, r2=0.1, bp_window_size=500000) # ~113 min to run  
mt_var_pru_filt = mt_var_filt.filter_rows(hl.is_defined(pruned[mt_var_filt.row_key])) 
print('Num of variants after LD pruning = ' + str(mt_var_pru_filt.count()[0])) # 248634 snps

#### 3c. Write out an intermediate file
The LD pruning step takes a non negligble time to run so to ensure that the downstream analyses steps don't take a very long time we write out an intermediate file. This write out step should take around 23 minutes to run. 

Due to the use of the apply_qc function however, you do not need to run through the write out step. Instead, the function will automatically read in the version of the dataset we wrote out when creating these tutorials. 

If the user wishes to export their own intermediate file, they can do so by changing the intermediate path and then replacing the apply_qc() function call with `hl.read_matrix_table(intermediate_path)`

In [ ]:
# # this step will take ~23 min
# # writing out an intermediate file to speed up subsequent analyses
# mt_var_pru_filt.write(intermediate_file_path, overwrite=False)

# Read the intermediate file back in for subsequent analyses
mt_var_pru_filt = apply_qc(ld_prune=True)

# 4. Run PC Relate   
<br>
<details><summary>Click <u><span style="color:blue">here</span></u> to learn why we are doing this. </summary>
<br>
When doing Principal Component Analysis (PCA), we need to separate the related and unrelated samples before computing the PC scores and ploting them. This is because if we compute PCA with the related samples in the data set, the population structure and clustering will be affected by the relatedness that exists among those samples. Thus, we first have to identify the related individuals by computing relatedness estimates (kinship statistic in this case) using a variant of the PC-Relate method in Hail. We used a minimum minor allele frequency (MAF) filter of 0.05, excluded sample pairs with kinship less than 0.05, and used 20 principal components (PC) to control for population structure. After getting the sample ID pairs for the related samples, we then separate the filtered and pruned mt into relateds and unrelateds.
   
    
For more information on relatedness click <a href="https://hail.is/docs/0.2/methods/relatedness.html#relatedness"> here</a>.
    
</details>

<br>
<details><summary> Click <u><span style="color:blue">here</span></u> to learn what metrics we used for pc_relate. 
 </summary>
    
<br>
We computed the kinship statistic using:
<ul>
<li>a minimum minor allele frequency filter of 0.05</li>
<li>excluding sample-pairs with kinship less than 0.05</li>
<li>20 principal components to control for population structure</li>
</ul>
    
For more information on the pc_relate method click <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4716688/">here</a>.
    
</details>

<br>
<details><summary> For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
    
<ul>
<li><a href="https://hail.is/docs/0.2/methods/relatedness.html#hail.methods.pc_relate"> More on  <i> pc_relate() </i></a></li>

<li><a href="https://hail.is/docs/0.2/methods/misc.html#hail.methods.maximal_independent_set"> More on  <i> maximal_independent_set() </i></a></li>
</ul>
    
</details>

[Back to Index](#Index)

In [ ]:
# Compute kinship statistic
# Takes ~4min to run
relatedness_ht = hl.pc_relate(
    mt_var_pru_filt.GT, min_individual_maf=0.05, min_kinship=0.05, statistics='kin', k=20).key_by() 

# Identify closely related individuals in pairs (list of sample IDs) 
# Takes ~2hr & 22min to run
related_samples_to_remove = hl.maximal_independent_set(relatedness_ht.i, relatedness_ht.j, False) 

# Subset the filtered and pruned mt to samples that are NOT present in the list of related individuals  
mt_unrel = mt_var_pru_filt.filter_cols(hl.is_defined(related_samples_to_remove[mt_var_pru_filt.col_key]), keep=False) 

# Do the same as above but this time subset to samples that are present in the related-individuals list   
mt_rel = mt_var_pru_filt.filter_cols(hl.is_defined(related_samples_to_remove[mt_var_pru_filt.col_key]), keep=True) 

In [ ]:
# # Write out the unrelated and related mts since they are used beyond this notebook in other analyses     
# # Unrelated mt
# mt_unrel.write(unrel_path, overwrite=False) 

# # Related mt 
# mt_rel.write(rel_path, overwrite=False)

In [ ]:
# Read the related and unrelated mts back in using apply_qc
# Unrelated mt
mt_unrel = apply_qc(rel_unrel='unrelated_pre_outlier') 

# Related mt 
mt_rel = apply_qc(rel_unrel='related_pre_outlier')

# 5. PCA
<br>
<details><summary>Click <u><span style="color:blue">here</span></u> to learn why we are doing this. </summary>
<br>
PCA is ran on the unrelated samples first. Then, the related samples are projected onto the PC space of the unrelated samples to get their PC scores. This way the population structure and clustering is not affected by the relatedness among samples.  
    
</details>

[Back to Index](#Index)

### 5a. Function to Run PCA on Unrelated Individuals

[Back to Index](#Index)

In [ ]:
def run_pca(mt: hl.MatrixTable, reg_name:str, out_path: str, overwrite: bool = False):
    """
    Runs PCA on a data set
    :param mt: data set to run PCA on
    :param reg_name: region name for saving output purposes
    :param out_path: path for where to save the outputs
    :return:
    """

    pca_evals, pca_scores, pca_loadings = hl.hwe_normalized_pca(mt.GT, k=20, compute_loadings=True)
    pca_mt = mt.annotate_rows(pca_af=hl.agg.mean(mt.GT.n_alt_alleles()) / 2)
    pca_loadings = pca_loadings.annotate(pca_af=pca_mt.rows()[pca_loadings.key].pca_af)
    pca_scores = pca_scores.transmute(**{f'PC{i}': pca_scores.scores[i - 1] for i in range(1, 21)})
    
    pca_scores.export(out_path + reg_name + '_scores.txt.bgz')  # save individual-level-genetic-region PCs
    pca_loadings.write(out_path + reg_name + '_loadings.ht', overwrite)  # save PCA loadings

### 5b. Function to Project Related Individuals
<br>
<details><summary> For troubleshooting information click <u><span style="color:blue">here</span></u>. </summary>

If this function is not working, make sure you used the  <code>--packages gnomad</code> argument when starting your cluster
    
</details>

[Back to Index](#Index)

In [ ]:
from gnomad.sample_qc.ancestry import *

def project_individuals(pca_loadings, project_mt, reg_name:str, out_path: str, overwrite: bool = False):
    """
    Project samples into predefined PCA space
    :param pca_loadings: existing PCA space of unrelated samples 
    :param project_mt: Matrix Table of related samples to project  
    :param reg_name: region name for saving output purposes
    :param out_path: path for where to save PCA projection outputs
    :return:
    """
    ht_projections = pc_project(project_mt, pca_loadings)  
    ht_projections = ht_projections.transmute(**{f'PC{i}': ht_projections.scores[i - 1] for i in range(1, 21)}) 
    ht_projections.export(out_path + reg_name + '_projected_scores.txt.bgz') # save output   

ModuleNotFoundError: No module named 'gnomad'

### 5c. Global PCA

<br>
<details><summary> Click <u><span style="color:blue">here</span></u> to learn why are we doing this.</summary>
<br>
    
To see the population structure and clustering on a continental level and contextualize the data globally.    
    
</details>

[Back to Index](#Index)

In [ ]:
# Run PCA on the unrelated samples
run_pca(mt_unrel, 'global', pca_preoutlier_path, False)  

# Read in the PCA loadings of the unrelated samples
loadings = hl.read_table(pca_preoutlier_path+'global_loadings.ht') 

# Project the related samples onto the unrelated-samples' PC space 
project_individuals(loadings, mt_rel, 'global', pca_preoutlier_path, False) 

### 5d. Subcontinental PCA 
<br>

When running the following section, the notebook might freeze after printing the log for EUR, AFR and AMR. If this happens, do not restart it. Let it run and follow the progress with the outputs being generated.  

When complete, check for the following in your specified output path:
- 21 total output files (3 for each region)

Once you have confirmed you have the desired output do the following:
1. Save close and halt the current notebook
2. Open a new session
3. Proceed to the next step (run project_relateds function)


<br>
<details><summary> Click <u><span style="color:blue">here</span></u> to learn why are we doing this. </summary>
<br>
    
To see the population structure and clustering on a subcontinental level and contextualize data within continental regions. This also helped us identify outliers which were removed later on.     

</details>
<br>

<details><summary> For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 

<ul>
<li><a href="more info https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca"> More on <i> hwe_normalized_pca() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_rows"> More on <i> annotate_rows() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.Table.html#hail.Table.annotate"> More on <i> annotate() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.Table.html#hail.Table.transmute"> More on <i> transmute() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.Table.html#hail.Table.export"> More on <i> export() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/experimental/index.html#hail.experimental.pc_project"> More on <i> pc_project() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.expr.Expression.html#hail.expr.Expression.collect"> More on <i> collect() </i></a></li>
</ul>
    
</details>

[Back to Index](#Index)

In [ ]:
# Obtain a list of the continental regions in the data set (used the unrelated data set since it had more samples) 
regions = mt_unrel['hgdp_tgp_meta']['Genetic']['region'].collect()
regions = list(dict.fromkeys(regions)) # convert into a list
# There are 7 regions: EUR, AFR, AMR, EAS, CSA, OCE, and MID

# Set argument values for PCA 
subcont_pca_prefix = pca_preoutlier_path+'subcont_pca/' # path for outputs 
overwrite = False

# For each region, run PCA on the unrelated samples (~27min to run)
for i in regions:  
    # filter the unrelateds per region
    subcont_unrel = mt_unrel.filter_cols(mt_unrel['hgdp_tgp_meta']['Genetic']['region'] == i)  
    run_pca(subcont_unrel, i, subcont_pca_prefix, overwrite)

In [ ]:
# For each region, project the related samples onto the unrelated-samples' PC space (~2min to run)
for i in regions:
    # Read in the PCA loadings of the unrelated samples for each region 
    loadings = hl.read_table(subcont_pca_prefix + i + '_loadings.ht') 
    
    # Filter the related mt per region 
    subcont_rel = mt_rel.filter_cols(mt_rel['hgdp_tgp_meta']['Genetic']['region'] == i)  
    
    # Project 
    project_individuals(loadings, subcont_rel, i, subcont_pca_prefix, overwrite) 

# 6. Outlier Removal

After plotting the PCs using R (link_the_plotting_Rmarkdown_here), 22 outliers were identified (complete_the_table)

| s | Genetic region | Population | Note |
| --- | --- | --- | -- |
| HG01880 | AFR | ACB | - |
| HG01881 | AFR | ACB | - |
| NA20274 | AFR | ASW | - |
| NA20299 | AFR | ASW | - |
| NA20314 | AFR | ASW | Clusters with AMR in global PCA | 
| HGDP00013 | CSA | Brahui | - |
| HGDP00029 | CSA | Brahui | - |
| HGDP00057 | CSA | Balochi | - | 
| HGDP00130 | CSA | Makrani | Closer to AFR than most CSA |
| HGDP00150 | CSA | Makrani | - |
| HGDP01298 | EAS | Uygur | - |
| HGDP01303 | EAS | Uygur | - |
| HGDP01300 | EAS | Uygur | - |
| LP6005443-DNA_B02 | EAS | Uygur | - |
| HG01628 | EUR | IBS | - |
| HG01629 | EUR | IBS | - |
| HG01630 | EUR | IBS | - |
| HG01694 | EUR | IBS | - |
| HG01696 | EUR | IBS | - |
| HGDP00621 | MID | Bedouin | Closer to AFR than most MID |
| HGDP01270 | MID | Mozabite | Closer to AFR than most MID |
| HGDP01271 | MID | Mozabite | Closer to AFR than most MID |

In [ ]:
# Read in the unrelated and related mts to remove outliers and rerun pca  
mt_unrel_unfiltered = apply_qc(rel_unrel='unrelated_pre_outlier') # unrelated mt
mt_rel_unfiltered = apply_qc(rel_unrel='related_pre_outlier') # related mt

# Read the outliers file into a list
with hl.utils.hadoop_open(outliers_path) as file: 
    outliers = [line.rstrip('\n') for line in file]
    
# Capture and broadcast the list as an expression
outliers_list = hl.literal(outliers)

# Remove the 22 outliers from both mts
mt_unrel = mt_unrel_unfiltered.filter_cols(~outliers_list.contains(mt_unrel_unfiltered['s']))
mt_rel = mt_rel_unfiltered.filter_cols(~outliers_list.contains(mt_rel_unfiltered['s']))

# Validity check 
print('Unrelated: Before outlier removal ' + 
      str(mt_unrel_unfiltered.count()[1]) + ' | After outlier removal ' + 
      str(mt_unrel.count()[1]))

print('Related: Before outlier removal: ' + 
      str(mt_rel_unfiltered.count()[1]) + ' | After outlier removal ' + 
      str(mt_rel.count()[1])) num_outliers = (mt_unrel_unfiltered.count()[1] - 
                                              mt_unrel.count()[1]) + (mt_rel_unfiltered.count()[1] - 
                                                                      mt_rel.count()[1])
print('Total samples removed = ' + str(num_outliers))

# 7. Rerun PCA

**Before running the sections below make sure you have run sections 5a (PCA) and 5b (projection) above.**

<br>
<details><summary> To learn what is different from the prior PCA run click <u><span style="color:blue">here</span></u>.</summary>
<ul>
<li>updated unrelated and related mts (outliers removed)</li>
<li>new paths for the outputs</li>  
    </ul>
</details>

[Back to Index](#Index)

### 7a. Global PCA (without outliers)

In [ ]:
# Run PCA on the unrelated samples  
run_pca(mt_unrel, 'global', pca_postoutlier_path, False)

# Read in the PCA loadings of the unrelated samples  
loadings = hl.read_table(pca_postoutlier_path+'global_loadings.ht') 

# Project the related samples onto the unrelated-samples' PC space 
project_individuals(loadings, mt_rel, 'global', pca_postoutlier_path, False) 

### 7b. Subcontinental PCA (without outliers)

When running the following section, the notebook might freeze after printing the log for EUR, AFR and AMR. If this happens, do not restart it. Let it run and follow the progress with the outputs being generated.  

When complete, check for the following in your specified output path:
- 21 total output files (3 for each region)

Once you have confirmed you have the desired output do the following:
1. Save close and halt the current notebook
2. Open a new session
3. Proceed to the next step (run project_relateds function)

<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 

<ul>
<li><a href="more info https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca"> More on <i> hwe_normalized_pca() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_rows"> More on <i> annotate_rows() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.Table.html#hail.Table.annotate"> More on <i> annotate() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.Table.html#hail.Table.transmute"> More on <i> transmute() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.Table.html#hail.Table.export"> More on <i> export() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/experimental/index.html#hail.experimental.pc_project"> More on <i> pc_project() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.expr.Expression.html#hail.expr.Expression.collect"> More on <i> collect() </i></a></li>
    </ul>
    
</details>

[Back to Index](#Index)

In [ ]:
# Set argument values for PCA 
subcont_pca_prefix = pca_postoutlier_path+'subcont_pca/' # path for outputs 
overwrite = False 


# For each region, run PCA on the unrelated samples (~26 min to run) 
# "regions" is a list containing the 7 continental regions in the data set from section 4d
for i in regions: 
    # filter the unrelateds per region
    subcont_unrel = mt_unrel.filter_cols(mt_unrel['hgdp_tgp_meta']['Genetic']['region'] == i)  
    run_pca(subcont_unrel, i, subcont_pca_prefix, overwrite)

In [ ]:
# For each region, project the related samples onto the unrelated-samples' PC space (~3min to run)
for i in regions:
    # Read in the PCA loadings of the unrelated samples for each region
    loadings = hl.read_table(subcont_pca_prefix + i + '_loadings.ht') 
    
    # Filter the relateds per region 
    subcont_rel = mt_rel.filter_cols(mt_rel['hgdp_tgp_meta']['Genetic']['region'] == i)  
    
    # Project 
    project_individuals(loadings, subcont_rel, i, subcont_pca_prefix, overwrite) 

# 8. Write Out Matrix Table 
[Back to Index](#Index)

```python3
# Write out mts of unrelated and related samples separately (post-outlier removal) 
# Unrelated mt
mt_unrel.write('gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/unrelated.mt',
                overwrite=False)
# Related mt
 mt_rel.write('gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/related.mt',
               overwrite=False)
```